# MS2DeepScore API – Quickstart

*In order to use omigami, you need an access token provided by DataRevenue.*
## Introduction

This is a short guide on how you can add GNPS library matches to your mass spectra by their Tanimoto Score - using Omigami's MS2DeepScore API.

The API uses a MS2DeepScore model which was trained on > 100,000 MS/MS spectra from GNPS. It prepares the input spectra and using a Siamese neural network predicts the molecular structural similarity (Tanimoto score) of the input spectra with the GNPS library spectra. It then returns you the top library matches for each of your spectra.

This notebook shows you how to:
1. Specify any MGF file with two spectra.
2. Run a library search through the MS2DeepScore API
3. Save the results as XLSX.

# 1. Load a MS/MS dataset (MGF format)

We'll use a small MS/MS dataset in the MGF format from [here](https://gnps-external.ucsd.edu/gnpslibrary/GNPS-COLLECTIONS-MISC.mgf).
You can also select any other from the [GNPS spectral library](https://gnps-external.ucsd.edu/gnpslibrary), or - of course - use your own.

*Note that you will need the Precursor_MZ field `PEPMASS` and the abundance pairs in your MGF file.*

In [1]:
# Load your own MS/MS dataset
path_to_mgf = 'GNPS-2-SPECTRA.mgf'

In [ ]:
# OR download a small MS/MS dataset from GNPS, in the same directory as this notebook
import urllib.request

url = 'https://gnps-external.ucsd.edu/gnpslibrary/GNPS-COLLECTIONS-MISC.mgf'

path_to_mgf = f'GNPS-COLLECTIONS-MISC.mgf' # use your prefered saving path here

urllib.request.urlretrieve(url, path_to_mgf)

# 2. Query for the best matches with MS2DeepScore

`MS2DeepScore` is a python wrapper which :
- Builds a json payload from the MGF file
- Calls the MS2DeepScores API
- Formats the prediction results into readable dataframes

____
- `n_best` sets the number of matches you'd like per spectrum (it is set to 10 by default).
- `include_metadata` selects the metadata fields that will be displayed in the resulting top matches.

In the results dataframes, the input spectra can be identified by their number in the dataframes index, which refers to their order in the MGF file.  
*i.e.* `matches of spectrum 1` gives the spectrum_id and MS2DeepScore scores of the library spectra matches, for the first spectrum in the MGF file.

For each spectrum in the MGF file, the library spectra matches are sorted according to their MS2DeepScore similarity score (best is first).   
The following information about the predicted spectra are returned :
- `score`, the MS2DeepScore similarity score between the input spectrum and the library spectrum
- `matches of spectrum #`, the spectrum_ID of the matched library spectra for the spectrum number # in the MGF file
- `metadata`, metadata fields chosen with `include_metadata` in the input

In [ ]:
from omigami import MS2DeepScore

In [ ]:
# Initialize the client with your user token
ms2deepscore = MS2DeepScore(token="Valide Omigami Token")

In [ ]:
# Run MS2DeepScore prediction
spectra_matches = ms2deepscore.match_spectra_from_path(
    path_to_mgf, 
    n_best=10, 
    ion_mode="positive" 
    include_metadata=["smiles", "compound_name"]
)

## 2.1 View results
A list of dataframes is returned. To look at a specific dataframe you can call :
```python
ms2deepscore_matches[i]  # 'i' refers to the (i+1)th spectrum in the MGF file input
```

In [ ]:
spectra_matches[1]

# 3. Save results

Execute the following cell to save the results in an Excel file. For readability, each dataframe is saved in its own csv file under the matches directory.

In [ ]:
import os
os.makedirs("matches", exist_ok=True)
for i, spectrum_matches in enumerate(spectra_matches):
    spectrum_matches.to_csv(f"matches/spectrum_{i}.csv")

# 4. Create Plots

The following cell will show you an example of how to visualize your matches.

In [ ]:
from omigami import MoleculePlotter
plotter = MoleculePlotter()
plotter.plot_molecule_structure_grid(spectra_matches[1], draw_indices=True, molecule_image_size=[600, 600], substructure_highlight="C(=O)")

### Creating a Plot using the ClassyFire APIm

In [ ]:
plotter.plot_classyfire_result(spectra_matches[1])

### Creating a Plot using the NP-Classfier API

In [ ]:
plotter.plot_NPclassifier_result(spectra_matches[1], color='orange')

____